In [67]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate model

In [68]:
tokenizer = AutoTokenizer.from_pretrained("kuzgunlar/electra-turkish-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("kuzgunlar/electra-turkish-sentiment-analysis")

# Encode and Calculate

In [69]:
def tahmin(text):
    tokens = tokenizer.encode(text, return_tensors = "pt")
    result = model(tokens)
    return int(torch.argmax(result.logits))

# Collect Review

## Load Reviewsinto DataFrame and Score

In [70]:
import numpy as np
import pandas as pd

In [71]:
from sklearn.preprocessing import LabelEncoder

In [72]:
le = LabelEncoder()

In [73]:
data = pd.read_csv("TurkceTwitterDuyguAnalizi.csv")

In [74]:
data.head()

,Unnamed: 0,Tweets,Tweets_duygu
0,0,sayın cumhurbaşkanım turizm için ertelendiğin...,negatif
1,1,bir kere ya bir kere gencinizi dinleyin biz y...,pozitif
2,2,o kadar çok mağduruz ki neye üzüleceğime şaşır...,negatif
3,3,bu sınav tarihini değiştirirken pedaloglara v...,negatif
4,4,turizm yılı gençlik geleceğinizi kurtarır ;,negatif


In [75]:
Tweets = data["Tweets"]
Tweets_duygu = data["Tweets_duygu"]

In [76]:
type(data["Tweets_duygu"][5])

str

In [77]:
Tweets_duygu = le.fit_transform(Tweets_duygu)

In [78]:
Tweets_duygu = pd.DataFrame(data = Tweets_duygu, index = range(len(Tweets_duygu)), columns = ["Tweets_duygu"])

In [79]:
data = pd.concat([Tweets, Tweets_duygu], axis = 1)

In [80]:
data.head()

,Tweets,Tweets_duygu
0,sayın cumhurbaşkanım turizm için ertelendiğin...,0
1,bir kere ya bir kere gencinizi dinleyin biz y...,1
2,o kadar çok mağduruz ki neye üzüleceğime şaşır...,0
3,bu sınav tarihini değiştirirken pedaloglara v...,0
4,turizm yılı gençlik geleceğinizi kurtarır ;,0


In [81]:
tahmin(data["Tweets"][1])

1

In [83]:
data["Tweets"][1]

' bir kere ya bir kere gencinizi dinleyin biz yanlış bir şey demiyoruz ki herkes için doğrusunu söylüyoruz ;'

In [84]:
data["sentiment"] = data["Tweets"].apply(lambda x: tahmin(x[:]))

In [85]:
data

,Tweets,Tweets_duygu,sentiment
0,sayın cumhurbaşkanım turizm için ertelendiğin...,0,0
1,bir kere ya bir kere gencinizi dinleyin biz y...,1,1
2,o kadar çok mağduruz ki neye üzüleceğime şaşır...,0,0
3,bu sınav tarihini değiştirirken pedaloglara v...,0,0
4,turizm yılı gençlik geleceğinizi kurtarır ;,0,1
...,...,...,...
4197,turizmle ilgili kalemlerde fiyatlar ne kadar y...,0,0
4198,türkiye de kaç rus turist koronadan öldü ;,0,1
4199,ve bakanlığı görüşmelerine başladık devletimi...,1,1
4200,turizmde yeni bir hikayeye ihtiyaç var ;,1,0
